In [5]:
import numpy as np
import pandas as pd

In [6]:
np.zeros(5)

array([ 0.,  0.,  0.,  0.,  0.])

In [37]:
training_data = pd.read_csv('datasets/tweets.csv', delimiter=',')
testing_data = pd.read_csv('datasets/tweets2.csv', delimiter=',')

clinton_training_data = training_data[training_data.handle == "HillaryClinton"]["text"].as_matrix()
trump_training_data = training_data[training_data.handle == "realDonaldTrump"]["text"].as_matrix()

clinton_testing_data = testing_data[testing_data.handle == "HillaryClinton"]["text"].as_matrix()
trump_testing_data = testing_data[testing_data.handle == "realDonaldTrump"]["text"].as_matrix()

for text in clinton_training_data:
    image = np.zeros((140, 36))
    words = text.lower().split()
    
    charCount = 0
    for word in words:
        if "t.co" in word:
            continue
        else:
            for char in word:
                index = None
                if char.isalpha():
                    index = ord(char) - ord('a')
                elif char.isdigit():
                    index = ord(char) - ord('0') + 26
                image[charCount, index] = 1
                charCount += 1
    print image
    break
                

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
